https://cseweb.ucsd.edu/~jmcauley/datasets.html#amazon_reviews

https://amazon-reviews-2023.github.io/

In [7]:
!pip install pymongo
!pip install pandas
!pip install bs4

In [8]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os


data_folder = "data"
response = requests.get("https://amazon-reviews-2023.github.io/")
soup  = BeautifulSoup(response.text, "html.parser")
file_links = soup.find_all(
    name = "a",
    attrs = {
        "href" : re.compile(r'.*https://mcauleylab.ucsd.edu/public_datasets/data/amazon_2023/raw/.*')
    }
)
# data = {}
# for file_link in file_links:
#     category = file_link.attrs["href"].split("/")[-1]
#     print(category)
#     df = pd.read_json(
#         path_or_buf = file_link.attrs["href"], 
#         compression = "gzip",
#         lines = True
#     ).assign(
#         Category = category
#     )
#     # .to_feather(
#     #     f"{data_folder}/{category}.ftr"
#     # )
#     data[category] = len(df)
#     print("Done")

In [9]:
df = pd.read_json(
    path_or_buf = file_links[0].attrs["href"], 
    compression = "gzip",
    lines = True
)

In [11]:
import pymongo


database_name = "Amazon_Reviews"
collection_name = "All_Beauty"

with pymongo.MongoClient("mongodb://localhost:27017/") as client:
    db = client[database_name]
    if collection_name not in db.list_collection_names():
        db.create_collection(
            name = collection_name, 
            timeseries = {
                'timeField' : 'timestamp' 
            }
        )
        
    db[collection_name].insert_many(
        df.to_dict(
            orient = "records"
        )
    )

In [58]:
database_name = "Amazon_Reviews"
collection_name = "All_Beauty"
batch_size = 10

rows = []
with pymongo.MongoClient("mongodb://localhost:27017/") as client:
    db = client[database_name]
    cursor  = db[collection_name].find(
        filter = {},
        limit = 90,
        batch_size = batch_size
    )
    previous_retrieved = batch_size
    for row in cursor:
        if cursor.retrieved != previous_retrieved:
            previous_retrieved = cursor.retrieved
            rows.clear()
                
        rows.append(row)

In [57]:
len(rows)

10